# Attack on the Auth, Replay Check
- 1000 per case

In [1]:
from tensorflow.keras.models import load_model

2025-03-05 11:14:16.437185: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 11:14:16.499857: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 11:14:16.500761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-05 11:14:17.505506: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
from datetime import datetime
import random
import zipfile

In [3]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Imports and defs

In [4]:
def vertorizing_png_imges(address):
  # Load the PNG image
  image = Image.open(address)

  # Convert the image to RGB mode
  image = image.convert('RGB')

  # Resize the image to match the input size expected by the CNN
  desired_width = 31
  desired_height = 20
  image = image.resize((desired_width, desired_height))

  # Convert the image to a NumPy array
  image_array = np.array(image)

  # Reshape the array to match the input shape expected by the CNN
  # image_array = image_array.reshape((1, desired_height, desired_width, 3))

  # Normalize the array
  image_array = image_array.astype('float32') / 255.0

  return image_array

In [5]:
from IPython.display import clear_output

def progress_bar(index, total_length, name_of_list):
    bar_length = 50

    # Calculate the percentage of completion
    percent_complete = (index / total_length) * 100

    # Clear the current cell's output
    clear_output(wait=True)

    print(name_of_list)

    # Print the progress bar
    print("[", end="")
    completed_blocks = int(bar_length * (percent_complete / 100))
    print("*" * completed_blocks, end="")
    print("-" * (bar_length - completed_blocks), end="]\n")

    # Print the progress in the format: index/total_length
    print(f"{index}/{total_length}")

In [6]:
def numerical_labels(labels_arr, dict_of_labels={}):
  # This function change the labels of train and test
  # data to numerical ones.
  # Note: for the test data we should pass the train
  # data labels

  if dict_of_labels == {}:
    unique_labels = np.unique(labels_arr)

    for i, value in enumerate(unique_labels):
        dict_of_labels[value] = i

  # print(dict_of_labels)

  num_lbls = []
  for lbl in labels_arr:
    num_lbls.append(dict_of_labels[lbl])

  num_lbls = np.array(num_lbls)

  return dict_of_labels, num_lbls

In [7]:
import numpy as np

def false_acceptance_rate(y_true, y_pred):
    """
    Computes the False Acceptance Rate (FAR).
    
    Args:
        y_true (array-like): Ground-truth user IDs.
        y_pred (array-like): Predicted user IDs.
    
    Returns:
        float: False Acceptance Rate.
    """
    false_accepts = np.sum((y_true != y_pred))  # Misclassified as another user
    total_attempts = len(y_true)
    return false_accepts / total_attempts

def false_rejection_rate(y_true, y_pred):
    """
    Computes the False Rejection Rate (FRR).
    
    Args:
        y_true (array-like): Ground-truth user IDs.
        y_pred (array-like): Predicted user IDs.
    
    Returns:
        float: False Rejection Rate.
    """
    false_rejections = np.sum((y_true == y_pred) == False)  # Misclassified cases
    total_attempts = len(y_true)
    return false_rejections / total_attempts

def attack_success_rate(y_true, y_pred):
    """
    Computes the overall Attack Success Rate (ASR).
    
    Args:
        y_true (array-like): Ground-truth user IDs.
        y_pred (array-like): Predicted user IDs.
    
    Returns:
        float: Attack Success Rate.
    """
    # Attack success = a sample is misclassified as another user (not itself)
    successful_attacks = np.sum((y_true != y_pred))
    total_attempts = len(y_true)
    return successful_attacks / total_attempts

def attack_success_rate_per_user(y_true, y_pred):
    """
    Computes Attack Success Rate per user.
    
    Args:
        y_true (array-like): Ground-truth user IDs.
        y_pred (array-like): Predicted user IDs.
    
    Returns:
        dict: Attack success rate for each user.
    """
    unique_users = np.unique(y_true)
    asr_per_user = {}

    for user in unique_users:
        user_mask = (y_true == user)  # Filter for this user's attempts
        user_attempts = np.sum(user_mask)
        user_successful_attacks = np.sum((y_true[user_mask] != y_pred[user_mask]))  # Misclassifications
        
        if user_attempts > 0:
            asr_per_user[user] = user_successful_attacks / user_attempts
        else:
            asr_per_user[user] = 0.0  # If no attempts, ASR is 0

    return asr_per_user

In [ ]:
import random
def creating_random_signal(len_signal):
    random.rand()

In [ ]:
from ishneholterlib import Holter
import biosignalsnotebooks as bsnb
from scipy.signal import detrend

In [9]:
# Getting users id
users_id = []
path = "../users_EKM_files_6sec_5bpf"
dirs = os.listdir(path)
for dir in dirs:
  user_id = dir.split("_")[-1]
  users_id.append(user_id)

dataset_path = "../users_EKM_files_6sec_5bpf"
users_files = os.listdir(dataset_path)

In [11]:
ecg200_dataset = "/home/sadeghi/Amirhossein/ECG200"
ranges = {}
for _user in users_id:
    ranges[_user] = {}

sum_x_max = 0
sum_y_max = 0 
sum_z_max = 0

sum_x_min = 0
sum_y_min = 0
sum_z_min = 0

for _file in os.listdir(ecg200_dataset):
    if _file.endswith(".ecg"):
        user = _file.split(".")[0]
        ecg_file = _file
        ecg_file_path = ecg200_dataset + "/" + ecg_file
        user_leads_all_data = Holter(ecg_file_path)
        user_leads_all_data.load_data()

        x_lead = user_leads_all_data.lead[0].data
        y_lead = user_leads_all_data.lead[1].data
        z_lead = user_leads_all_data.lead[2].data

        max_x_lead = max(x_lead) 
        max_y_lead = max(y_lead)
        max_z_lead = max(z_lead)

        min_x_lead = min(x_lead)
        min_y_lead = min(y_lead)
        min_z_lead = min(z_lead)

        ranges[user]["x_lead"] = [max_x_lead, min_x_lead] 
        ranges[user]["y_lead"] = [max_y_lead, min_y_lead] 
        ranges[user]["z_lead"] = [max_z_lead, min_z_lead]

        sum_x_max += max_x_lead
        sum_y_max += max_y_lead 
        sum_z_max += max_z_lead
    
        sum_x_min += min_x_lead
        sum_y_min += min_y_lead
        sum_z_min += min_z_lead


In [ ]:
print(f"Average max x_lead: {max_x_lead / 199}")
print(f"Average max y_lead: {max_y_lead / 199}")
print(f"Average max z_lead: {max_z_lead / 199}")
print()
print(f"Average min x_lead{min_x_lead / 199}")
print(f"Average min y_lead{min_y_lead / 199}")
print(f"Average min z_lead{min_z_lead / 199}")

# Attack on: Omni_RI 5bpf, 6seconds

In [115]:
# Load the keras model
omniRI_5bpf_6sec_model = load_model("./5bpf_6sec_inBoundary_1000_auth_6000_Omni_RI.keras")

In [116]:
# Getting users id
users_id = []
path = "../users_EKM_files_6sec_5bpf"
dirs = os.listdir(path)
for dir in dirs:
  user_id = dir.split("_")[-1]
  users_id.append(user_id)

dataset_path = "../users_EKM_files_6sec_5bpf"
users_files = os.listdir(dataset_path)

In [117]:
# Selecting N amount of EKMs for each user
users_bpf_EKMs_x = []
users_bpf_EKMs_y = []

N_each_user_ekms_amount = 400
dataset_path = "../users_EKM_files_6sec_5bpf"

for _files in users_files:
    user_id = _files.split("_")[-1]
    if user_id in users_id:
      for _ekm_number in range(N_each_user_ekms_amount):
        # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
        users_bpf_EKMs_x.append(f"{dataset_path}/{user_id}/y_lead/5bpf-ekm-bpf_recording_signal_length_ekm_dataset-{user_id}-{_ekm_number}.png")
        users_bpf_EKMs_y.append(user_id)

In [118]:
# Vectorizing EKMs of users
for index, ekm_path in enumerate(users_bpf_EKMs_x):
  # progress_bar(index, len(users_bpf_EKMs_x), "Vertorizing")
  users_bpf_EKMs_x[index] = vertorizing_png_imges(ekm_path)

In [119]:
label_dict, users_bpf_EKMs_y = numerical_labels(users_bpf_EKMs_y)

In [120]:
users_bpf_EKMs_x = np.array(users_bpf_EKMs_x)
users_bpf_EKMs_y = np.array(users_bpf_EKMs_y)

In [121]:
y_prob = omniRI_5bpf_6sec_model.predict(users_bpf_EKMs_x)

2488/2488 [==============================] - 18s 7ms/step


## Calculating attack measurements (ASR: Attack Success Rate)

In [127]:
# far = false_acceptance_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))  # Assuming softmax output
# frr = false_rejection_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))
asr = attack_success_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))
asr_per_user = attack_success_rate_per_user(users_bpf_EKMs_y, y_prob.argmax(axis=1))

# Print results
# print(f"False Acceptance Rate (FAR): {far:.6f}")
# print(f"False Rejection Rate (FRR): {frr:.6f}")
print(f"Attack Success Rate (ASR): {asr*100:.4f}%")

print("\nAttack Success Rate per User:")
max = 0
max_id = ""
min = 100
min_id = ""

for user, rate in asr_per_user.items():
    if max < rate:
        max = rate
        max_id = user
    if min > rate:
        min = rate
        min_id = user
    print(f"User {user}: {rate*100:.4f}%")

Attack Success Rate (ASR): 1.2462%

Attack Success Rate per User:
User 0: 1.0000%
User 1: 4.0000%
User 2: 0.2500%
User 3: 0.5000%
User 4: 1.2500%
User 5: 0.0000%
User 6: 0.2500%
User 7: 0.5000%
User 8: 1.0000%
User 9: 3.0000%
User 10: 8.0000%
User 11: 0.0000%
User 12: 0.5000%
User 13: 1.7500%
User 14: 6.2500%
User 15: 1.0000%
User 16: 3.0000%
User 17: 1.0000%
User 18: 0.2500%
User 19: 0.0000%
User 20: 0.7500%
User 21: 1.5000%
User 22: 4.2500%
User 23: 2.2500%
User 24: 1.0000%
User 25: 0.0000%
User 26: 0.7500%
User 27: 0.2500%
User 28: 0.0000%
User 29: 0.0000%
User 30: 0.0000%
User 31: 0.0000%
User 32: 0.2500%
User 33: 0.0000%
User 34: 0.2500%
User 35: 1.7500%
User 36: 0.2500%
User 37: 1.7500%
User 38: 0.7500%
User 39: 4.5000%
User 40: 0.0000%
User 41: 0.0000%
User 42: 0.2500%
User 43: 0.2500%
User 44: 0.2500%
User 45: 9.7500%
User 46: 0.0000%
User 47: 1.2500%
User 48: 0.7500%
User 49: 0.0000%
User 50: 0.5000%
User 51: 1.7500%
User 52: 2.2500%
User 53: 7.0000%
User 54: 0.2500%
User 55: 

In [128]:
print(f"Max success attack: user {max_id}: {max*100}%")
print(f"Min success attack: user {min_id}: {min*100}%")

Max success attack: user 45: 9.75%
Min success attack: user 5: 0.0%


# Attack on: Mono_RI 5bpf, 6seconds

In [129]:
# Load the keras model
monoRI_5bpf_6sec_model = load_model("./5bpf_6sec_inBoundary_1000_auth_6000_Mono_RI_user1027.keras")

In [130]:
# Getting users id
users_id = []
path = "../users_EKM_files_6sec_5bpf"
dirs = os.listdir(path)
for dir in dirs:
  user_id = dir.split("_")[-1]
  users_id.append(user_id)

dataset_path = "../users_EKM_files_6sec_5bpf"
users_files = os.listdir(dataset_path)

In [131]:
# Creating dict of counters for each randomly-chosen users
# of ecg 200 dataset
# Also creating dict of users' EKMs
ecg_200_users_EKM_amount_dict = {}
ecg_200_users_EKMs_dict = {}

for user in users_id:
  ecg_200_users_EKM_amount_dict[user] = 0
  ecg_200_users_EKMs_dict[user] = []

In [132]:
# Counting each user's EKMs in dataset and collecting EKMs of him/her
dataset_path = "../users_EKM_files_6sec_5bpf"
users_files = os.listdir(dataset_path)

for _files in users_files:
    user_id = _files.split("_")[-1]
    if user_id in users_id:
      ecg_200_users_EKM_amount_dict[user_id] = len(os.listdir(f"{dataset_path}/{user_id}/y_lead"))
      ecg_200_users_EKMs_dict[user_id] = os.listdir(f"{dataset_path}/{user_id}/y_lead")

In [133]:
# Function to extract the number at the end of the string
import re
def extract_number(s): 
  match = re.search(r'-(\d+)\.png$', s) 
  return int(match.group(1)) if match else -1

In [134]:
dataset_path = "../users_EKM_files_6sec_5bpf"

In [135]:
def one_vs_all_EKMs(user_id):
    '''
    Gets the id of user and returns the X and y for the model
    '''

    # Selecting N amount of EKMs for the random user
    users_bpf_EKMs_x = []
    users_bpf_EKMs_y = []

    N_each_user_ekms_amount = 400

    if user_id in users_id:
        userid_ekms = ecg_200_users_EKMs_dict[user_id][:N_each_user_ekms_amount]
        
        # Sort the array based on the extracted number 
        sorted_userid_ekms = sorted(userid_ekms, key=extract_number)
        
        if len(sorted_userid_ekms) < N_each_user_ekms_amount:
            print(user_id, len(sorted_userid_ekms))
        
        for _ekm in sorted_userid_ekms:
            # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
            users_bpf_EKMs_x.append(f"{dataset_path}/{user_id}/y_lead/{_ekm}")
            users_bpf_EKMs_y.append(1)

    # Selecting N amount of EKMs for the rest of users
    rest_users = [_ for _ in users_files if _ != user_id]
    N_rest_user_ekms_amount = 8  # 400 / 199 => 2

    for _files in rest_users[:50]:
        rest_user_id = _files.split("_")[-1]
        if rest_user_id in users_id:
            userid_ekms = ecg_200_users_EKMs_dict[rest_user_id][:N_rest_user_ekms_amount]
            # Sort the array based on the extracted number 
            sorted_userid_ekms = sorted(userid_ekms, key=extract_number)
            
            if len(sorted_userid_ekms) < N_rest_user_ekms_amount:
                print(rest_user_id, len(sorted_userid_ekms))
            
            for _ekm in sorted_userid_ekms:
                # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
                users_bpf_EKMs_x.append(f"{dataset_path}/{rest_user_id}/y_lead/{_ekm}")
                users_bpf_EKMs_y.append(0)
    
    return users_bpf_EKMs_x, users_bpf_EKMs_y

In [136]:
users_bpf_EKMs_x, users_bpf_EKMs_y = one_vs_all_EKMs("1027")

In [137]:
# Vectorizing EKMs of users
for index, ekm_path in enumerate(users_bpf_EKMs_x):
  # progress_bar(index, len(users_bpf_EKMs_x), "Vertorizing")
  users_bpf_EKMs_x[index] = vertorizing_png_imges(ekm_path)

In [138]:
users_bpf_EKMs_x = np.array(users_bpf_EKMs_x)
users_bpf_EKMs_y = np.array(users_bpf_EKMs_y)

In [139]:
y_prob = monoRI_5bpf_6sec_model.predict(users_bpf_EKMs_x)

25/25 [==============================] - 0s 7ms/step


In [140]:
asr = attack_success_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))
asr_per_user = attack_success_rate_per_user(users_bpf_EKMs_y, y_prob.argmax(axis=1))

# Print results
print(f"Attack Success Rate (ASR): {asr*100:.4f}%")

print("\nAttack Success Rate per User:")
max = 0
max_id = ""
min = 100
min_id = ""

for user, rate in asr_per_user.items():
    if max < rate:
        max = rate
        max_id = user
    if min > rate:
        min = rate
        min_id = user
    print(f"User {user}: {rate*100:.4f}%")

Attack Success Rate (ASR): 0.3750%

Attack Success Rate per User:
User 0: 0.5000%
User 1: 0.2500%


In [143]:
print(f"Max success attack: user {max_id}: {max*100}%")
print(f"Min success attack: user {min_id}: {min*100}%")

Max success attack: user 0: 0.5%
Min success attack: user 1: 0.25%


# Attack on: Omni_RI 5bpf, 5seconds

In [144]:
# Load the keras model
omniRI_5bpf_5sec_model = load_model("./5bpf_5sec_inBoundary_1000_auth_6000_Omni_RI.keras")

In [145]:
# Getting users id
users_id = []
path = "../users_EKM_files_5sec_5bpf"
dirs = os.listdir(path)
for dir in dirs:
  user_id = dir.split("_")[-1]
  users_id.append(user_id)

dataset_path = "../users_EKM_files_5sec_5bpf"
users_files = os.listdir(dataset_path)

In [146]:
# Selecting N amount of EKMs for each user
users_bpf_EKMs_x = []
users_bpf_EKMs_y = []

N_each_user_ekms_amount = 400
dataset_path = "../users_EKM_files_5sec_5bpf"

for _files in users_files:
    user_id = _files.split("_")[-1]
    if user_id in users_id:
      for _ekm_number in range(N_each_user_ekms_amount):
        # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
        users_bpf_EKMs_x.append(f"{dataset_path}/{user_id}/y_lead/5bpf-ekm-bpf_recording_signal_length_ekm_dataset-{user_id}-{_ekm_number}.png")
        users_bpf_EKMs_y.append(user_id)

In [147]:
# Vectorizing EKMs of users
for index, ekm_path in enumerate(users_bpf_EKMs_x):
  # progress_bar(index, len(users_bpf_EKMs_x), "Vertorizing")
  users_bpf_EKMs_x[index] = vertorizing_png_imges(ekm_path)

In [148]:
label_dict, users_bpf_EKMs_y = numerical_labels(users_bpf_EKMs_y)

In [149]:
users_bpf_EKMs_x = np.array(users_bpf_EKMs_x)
users_bpf_EKMs_y = np.array(users_bpf_EKMs_y)

In [150]:
y_prob = omniRI_5bpf_5sec_model.predict(users_bpf_EKMs_x)

2488/2488 [==============================] - 18s 7ms/step


In [151]:
asr = attack_success_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))
asr_per_user = attack_success_rate_per_user(users_bpf_EKMs_y, y_prob.argmax(axis=1))

# Print results
print(f"Attack Success Rate (ASR): {asr*100:.4f}%")

print("\nAttack Success Rate per User:")
max = 0
max_id = ""
min = 100
min_id = ""

for user, rate in asr_per_user.items():
    if max < rate:
        max = rate
        max_id = user
    if min > rate:
        min = rate
        min_id = user
    print(f"User {user}: {rate*100:.4f}%")

Attack Success Rate (ASR): 5.4862%

Attack Success Rate per User:
User 0: 3.0000%
User 1: 19.0000%
User 2: 1.0000%
User 3: 4.0000%
User 4: 0.5000%
User 5: 0.7500%
User 6: 0.5000%
User 7: 0.7500%
User 8: 9.5000%
User 9: 9.2500%
User 10: 15.5000%
User 11: 3.7500%
User 12: 4.5000%
User 13: 13.5000%
User 14: 14.7500%
User 15: 4.2500%
User 16: 4.0000%
User 17: 1.2500%
User 18: 1.5000%
User 19: 0.2500%
User 20: 2.0000%
User 21: 3.2500%
User 22: 18.5000%
User 23: 7.0000%
User 24: 2.2500%
User 25: 0.2500%
User 26: 3.2500%
User 27: 0.5000%
User 28: 0.5000%
User 29: 4.0000%
User 30: 2.5000%
User 31: 1.7500%
User 32: 6.2500%
User 33: 0.2500%
User 34: 2.2500%
User 35: 6.7500%
User 36: 2.0000%
User 37: 1.2500%
User 38: 1.2500%
User 39: 16.7500%
User 40: 0.0000%
User 41: 1.0000%
User 42: 1.2500%
User 43: 0.7500%
User 44: 0.2500%
User 45: 22.0000%
User 46: 0.0000%
User 47: 0.7500%
User 48: 3.2500%
User 49: 0.2500%
User 50: 4.7500%
User 51: 7.0000%
User 52: 23.0000%
User 53: 26.0000%
User 54: 0.5000%


In [152]:
print(f"Max success attack: user {max_id}: {max*100}%")
print(f"Min success attack: user {min_id}: {min*100}%")

Max success attack: user 97: 53.0%
Min success attack: user 40: 0.0%


# Attack on: Mono_RI 5bpf, 7seconds

In [153]:
# Load the keras model
monoRI_5bpf_7sec_model = load_model("./5bpf_7sec_inBoundary_1000_auth_6000_Mono_RI_user1027.keras")

In [154]:
# Getting users id
users_id = []
path = "../users_EKM_files_7sec_5bpf"
dirs = os.listdir(path)
for dir in dirs:
  user_id = dir.split("_")[-1]
  users_id.append(user_id)

dataset_path = "../users_EKM_files_7sec_5bpf"
users_files = os.listdir(dataset_path)

In [155]:
# Creating dict of counters for each randomly-chosen users
# of ecg 200 dataset
# Also creating dict of users' EKMs
ecg_200_users_EKM_amount_dict = {}
ecg_200_users_EKMs_dict = {}

for user in users_id:
  ecg_200_users_EKM_amount_dict[user] = 0
  ecg_200_users_EKMs_dict[user] = []

In [156]:
# Counting each user's EKMs in dataset and collecting EKMs of him/her
dataset_path = "../users_EKM_files_7sec_5bpf"
users_files = os.listdir(dataset_path)

for _files in users_files:
    user_id = _files.split("_")[-1]
    if user_id in users_id:
      ecg_200_users_EKM_amount_dict[user_id] = len(os.listdir(f"{dataset_path}/{user_id}/y_lead"))
      ecg_200_users_EKMs_dict[user_id] = os.listdir(f"{dataset_path}/{user_id}/y_lead")

In [157]:
# Function to extract the number at the end of the string
import re
def extract_number(s): 
  match = re.search(r'-(\d+)\.png$', s) 
  return int(match.group(1)) if match else -1

In [158]:
dataset_path = "../users_EKM_files_7sec_5bpf"

In [159]:
def one_vs_all_EKMs(user_id):
    '''
    Gets the id of user and returns the X and y for the model
    '''

    # Selecting N amount of EKMs for the random user
    users_bpf_EKMs_x = []
    users_bpf_EKMs_y = []

    N_each_user_ekms_amount = 400

    if user_id in users_id:
        userid_ekms = ecg_200_users_EKMs_dict[user_id][:N_each_user_ekms_amount]
        
        # Sort the array based on the extracted number 
        sorted_userid_ekms = sorted(userid_ekms, key=extract_number)
        
        if len(sorted_userid_ekms) < N_each_user_ekms_amount:
            print(user_id, len(sorted_userid_ekms))
        
        for _ekm in sorted_userid_ekms:
            # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
            users_bpf_EKMs_x.append(f"{dataset_path}/{user_id}/y_lead/{_ekm}")
            users_bpf_EKMs_y.append(1)

    # Selecting N amount of EKMs for the rest of users
    rest_users = [_ for _ in users_files if _ != user_id]
    N_rest_user_ekms_amount = 8  # 400 / 199 => 2

    for _files in rest_users[:50]:
        rest_user_id = _files.split("_")[-1]
        if rest_user_id in users_id:
            userid_ekms = ecg_200_users_EKMs_dict[rest_user_id][:N_rest_user_ekms_amount]
            # Sort the array based on the extracted number 
            sorted_userid_ekms = sorted(userid_ekms, key=extract_number)
            
            if len(sorted_userid_ekms) < N_rest_user_ekms_amount:
                print(rest_user_id, len(sorted_userid_ekms))
            
            for _ekm in sorted_userid_ekms:
                # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
                users_bpf_EKMs_x.append(f"{dataset_path}/{rest_user_id}/y_lead/{_ekm}")
                users_bpf_EKMs_y.append(0)
    
    return users_bpf_EKMs_x, users_bpf_EKMs_y

In [160]:
users_bpf_EKMs_x, users_bpf_EKMs_y = one_vs_all_EKMs("1027")

In [162]:
# Vectorizing EKMs of users
for index, ekm_path in enumerate(users_bpf_EKMs_x):
  # progress_bar(index, len(users_bpf_EKMs_x), "Vertorizing")
  users_bpf_EKMs_x[index] = vertorizing_png_imges(ekm_path)

In [163]:
users_bpf_EKMs_x = np.array(users_bpf_EKMs_x)
users_bpf_EKMs_y = np.array(users_bpf_EKMs_y)

In [164]:
y_prob = monoRI_5bpf_6sec_model.predict(users_bpf_EKMs_x)

25/25 [==============================] - 0s 8ms/step


In [165]:
asr = attack_success_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))
asr_per_user = attack_success_rate_per_user(users_bpf_EKMs_y, y_prob.argmax(axis=1))

# Print results
print(f"Attack Success Rate (ASR): {asr*100:.4f}%")

print("\nAttack Success Rate per User:")
max = 0
max_id = ""
min = 100
min_id = ""

for user, rate in asr_per_user.items():
    if max < rate:
        max = rate
        max_id = user
    if min > rate:
        min = rate
        min_id = user
    print(f"User {user}: {rate*100:.4f}%")

Attack Success Rate (ASR): 2.6250%

Attack Success Rate per User:
User 0: 2.5000%
User 1: 2.7500%


In [166]:
print(f"Max success attack: user {max_id}: {max*100}%")
print(f"Min success attack: user {min_id}: {min*100}%")

Max success attack: user 1: 2.75%
Min success attack: user 0: 2.5%


# Attack on: Omni_RI 3bpf, 3seconds

In [172]:
# Load the keras model
omniRI_3bpf_3sec_model = load_model("./3bpf_3sec_inBoundary_1000_auth_6000_Omni_RI.keras")

In [173]:
# Getting users id
users_id = []
path = "../users_EKM_files_3sec_3bpf"
dirs = os.listdir(path)
for dir in dirs:
  user_id = dir.split("_")[-1]
  users_id.append(user_id)

dataset_path = "../users_EKM_files_3sec_3bpf"
users_files = os.listdir(dataset_path)

In [176]:
# Selecting N amount of EKMs for each user
users_bpf_EKMs_x = []
users_bpf_EKMs_y = []

N_each_user_ekms_amount = 400
dataset_path = "../users_EKM_files_3sec_3bpf"

for _files in users_files:
    user_id = _files.split("_")[-1]
    if user_id in users_id:
      for _ekm_number in range(N_each_user_ekms_amount):
        # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
        users_bpf_EKMs_x.append(f"{dataset_path}/{user_id}/y_lead/3bpf-ekm-bpf_recording_signal_length_ekm_dataset-{user_id}-{_ekm_number}.png")
        users_bpf_EKMs_y.append(user_id)

In [177]:
# Vectorizing EKMs of users
for index, ekm_path in enumerate(users_bpf_EKMs_x):
  # progress_bar(index, len(users_bpf_EKMs_x), "Vertorizing")
  users_bpf_EKMs_x[index] = vertorizing_png_imges(ekm_path)

In [178]:
label_dict, users_bpf_EKMs_y = numerical_labels(users_bpf_EKMs_y)

In [179]:
users_bpf_EKMs_x = np.array(users_bpf_EKMs_x)
users_bpf_EKMs_y = np.array(users_bpf_EKMs_y)

In [180]:
y_prob = omniRI_3bpf_3sec_model.predict(users_bpf_EKMs_x)

2488/2488 [==============================] - 18s 7ms/step


In [181]:
asr = attack_success_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))
asr_per_user = attack_success_rate_per_user(users_bpf_EKMs_y, y_prob.argmax(axis=1))

# Print results
print(f"Attack Success Rate (ASR): {asr*100:.4f}%")

print("\nAttack Success Rate per User:")
max = 0
max_id = ""
min = 100
min_id = ""

for user, rate in asr_per_user.items():
    if max < rate:
        max = rate
        max_id = user
    if min > rate:
        min = rate
        min_id = user
    print(f"User {user}: {rate*100:.4f}%")

Attack Success Rate (ASR): 1.9523%

Attack Success Rate per User:
User 0: 1.2500%
User 1: 9.5000%
User 2: 0.5000%
User 3: 5.2500%
User 4: 0.2500%
User 5: 1.2500%
User 6: 0.5000%
User 7: 1.5000%
User 8: 0.5000%
User 9: 0.2500%
User 10: 4.7500%
User 11: 0.5000%
User 12: 0.2500%
User 13: 3.2500%
User 14: 12.5000%
User 15: 1.0000%
User 16: 5.0000%
User 17: 0.5000%
User 18: 2.0000%
User 19: 0.0000%
User 20: 1.2500%
User 21: 2.0000%
User 22: 0.0000%
User 23: 4.7500%
User 24: 0.0000%
User 25: 0.5000%
User 26: 0.2500%
User 27: 1.0000%
User 28: 0.0000%
User 29: 0.0000%
User 30: 2.0000%
User 31: 0.7500%
User 32: 4.0000%
User 33: 0.7500%
User 34: 2.0000%
User 35: 2.2500%
User 36: 1.0000%
User 37: 2.7500%
User 38: 3.2500%
User 39: 4.5000%
User 40: 0.2500%
User 41: 0.0000%
User 42: 0.7500%
User 43: 0.5000%
User 44: 0.2500%
User 45: 11.5000%
User 46: 0.2500%
User 47: 0.5000%
User 48: 0.7500%
User 49: 0.5000%
User 50: 2.0000%
User 51: 0.2500%
User 52: 5.0000%
User 53: 15.2500%
User 54: 0.2500%
User 5

In [182]:
print(f"Max success attack: user {max_id}: {max*100}%")
print(f"Min success attack: user {min_id}: {min*100}%")

Max success attack: user 53: 15.25%
Min success attack: user 19: 0.0%


# Attack on: Mono_RI 3bpf, 3seconds

In [183]:
# Load the keras model
monoRI_3bpf_3sec_model = load_model("./3bpf_3sec_inBoundary_1000_auth_6000_Mono_RI_user1027.keras")

In [184]:
# Getting users id
users_id = []
path = "../users_EKM_files_3sec_3bpf"
dirs = os.listdir(path)
for dir in dirs:
  user_id = dir.split("_")[-1]
  users_id.append(user_id)

dataset_path = "../users_EKM_files_3sec_3bpf"
users_files = os.listdir(dataset_path)

In [185]:
# Creating dict of counters for each randomly-chosen users
# of ecg 200 dataset
# Also creating dict of users' EKMs
ecg_200_users_EKM_amount_dict = {}
ecg_200_users_EKMs_dict = {}

for user in users_id:
  ecg_200_users_EKM_amount_dict[user] = 0
  ecg_200_users_EKMs_dict[user] = []

In [186]:
# Counting each user's EKMs in dataset and collecting EKMs of him/her
dataset_path = "../users_EKM_files_3sec_3bpf"
users_files = os.listdir(dataset_path)

for _files in users_files:
    user_id = _files.split("_")[-1]
    if user_id in users_id:
      ecg_200_users_EKM_amount_dict[user_id] = len(os.listdir(f"{dataset_path}/{user_id}/y_lead"))
      ecg_200_users_EKMs_dict[user_id] = os.listdir(f"{dataset_path}/{user_id}/y_lead")

In [187]:
# Function to extract the number at the end of the string
import re
def extract_number(s): 
  match = re.search(r'-(\d+)\.png$', s) 
  return int(match.group(1)) if match else -1

In [188]:
dataset_path = "../users_EKM_files_3sec_3bpf"

In [189]:
def one_vs_all_EKMs(user_id):
    '''
    Gets the id of user and returns the X and y for the model
    '''

    # Selecting N amount of EKMs for the random user
    users_bpf_EKMs_x = []
    users_bpf_EKMs_y = []

    N_each_user_ekms_amount = 400

    if user_id in users_id:
        userid_ekms = ecg_200_users_EKMs_dict[user_id][:N_each_user_ekms_amount]
        
        # Sort the array based on the extracted number 
        sorted_userid_ekms = sorted(userid_ekms, key=extract_number)
        
        if len(sorted_userid_ekms) < N_each_user_ekms_amount:
            print(user_id, len(sorted_userid_ekms))
        
        for _ekm in sorted_userid_ekms:
            # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
            users_bpf_EKMs_x.append(f"{dataset_path}/{user_id}/y_lead/{_ekm}")
            users_bpf_EKMs_y.append(1)

    # Selecting N amount of EKMs for the rest of users
    rest_users = [_ for _ in users_files if _ != user_id]
    N_rest_user_ekms_amount = 8  # 400 / 199 => 2

    for _files in rest_users[:50]:
        rest_user_id = _files.split("_")[-1]
        if rest_user_id in users_id:
            userid_ekms = ecg_200_users_EKMs_dict[rest_user_id][:N_rest_user_ekms_amount]
            # Sort the array based on the extracted number 
            sorted_userid_ekms = sorted(userid_ekms, key=extract_number)
            
            if len(sorted_userid_ekms) < N_rest_user_ekms_amount:
                print(rest_user_id, len(sorted_userid_ekms))
            
            for _ekm in sorted_userid_ekms:
                # 5bpf-ekm-bpf_recording_signal_length_ekm_dataset-2005-1008.png
                users_bpf_EKMs_x.append(f"{dataset_path}/{rest_user_id}/y_lead/{_ekm}")
                users_bpf_EKMs_y.append(0)
    
    return users_bpf_EKMs_x, users_bpf_EKMs_y

In [190]:
users_bpf_EKMs_x, users_bpf_EKMs_y = one_vs_all_EKMs("1027")

In [191]:
# Vectorizing EKMs of users
for index, ekm_path in enumerate(users_bpf_EKMs_x):
  # progress_bar(index, len(users_bpf_EKMs_x), "Vertorizing")
  users_bpf_EKMs_x[index] = vertorizing_png_imges(ekm_path)

In [192]:
users_bpf_EKMs_x = np.array(users_bpf_EKMs_x)
users_bpf_EKMs_y = np.array(users_bpf_EKMs_y)

In [193]:
y_prob = monoRI_3bpf_3sec_model.predict(users_bpf_EKMs_x)

25/25 [==============================] - 0s 8ms/step


In [194]:
asr = attack_success_rate(users_bpf_EKMs_y, y_prob.argmax(axis=1))
asr_per_user = attack_success_rate_per_user(users_bpf_EKMs_y, y_prob.argmax(axis=1))

# Print results
print(f"Attack Success Rate (ASR): {asr*100:.4f}%")

print("\nAttack Success Rate per User:")
max = 0
max_id = ""
min = 100
min_id = ""

for user, rate in asr_per_user.items():
    if max < rate:
        max = rate
        max_id = user
    if min > rate:
        min = rate
        min_id = user
    print(f"User {user}: {rate*100:.4f}%")

Attack Success Rate (ASR): 0.7500%

Attack Success Rate per User:
User 0: 0.7500%
User 1: 0.7500%


In [195]:
print(f"Max success attack: user {max_id}: {max*100}%")
print(f"Min success attack: user {min_id}: {min*100}%")

Max success attack: user 0: 0.75%
Min success attack: user 0: 0.75%


# Attack on: Omni_RI 3bpf, 4seconds

In [203]:
omniRI_3bpf_4sec_model = load_model("./3bpf_4sec_inBoundary_1000_auth_6000_Omni_RI.keras")

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 20, 31, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']